In [ ]:
import pandas as pd
import sys
sys.path.insert(0, '../')
from cleaner import Cleaner, Corretor
from tqdm.auto import tqdm

In [ ]:
dados = pd.read_pickle('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/MaskedLanguageModeling/LipSet/LipSet.pkl')
dados

In [ ]:
dados = dados.select_dtypes(include=['object'])
dados.dropna(inplace=True)
dados.drop_duplicates(inplace=True)
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
def executaLimpeza(dataframe: pd.DataFrame, column: str, cased: bool, accents: bool) -> pd.DataFrame:
    """
    Executa a limpeza do DataFrame usando as classes Cleaner e Corretor.
    """
    sentencesDrop = []
    for indice in tqdm(dataframe.index, desc="Executando a Limpeza", colour='yellow'):
        texto = dataframe[column][indice]
        if isinstance(texto, str):
            texto = Cleaner().clear(texto)
            texto = Corretor(cased, accents).corrige_termos(texto)
            dataframe.at[indice, column] = texto
        else:
            sentencesDrop.append(indice)
    dataframe = dataframe.drop(sentencesDrop)
    return dataframe

In [ ]:
lista_texts = []
for indice in tqdm(dados.index):
    text = dados.loc[indice, 'text'].split()[200:350]
    lista_texts.append(' '.join(text))
dados = pd.DataFrame({'text': lista_texts})
dados 

In [ ]:
dados = executaLimpeza(dados, 'text', cased=False, accents=False)
dados

In [ ]:
dados = dados.select_dtypes(include=['object'])
dados.dropna(inplace=True)
dados.drop_duplicates(inplace=True)
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
dados['tamanho'] = dados['text'].apply(lambda x: len(x.split()))
dados['tamanho'].describe()

In [ ]:
dados = dados[dados['tamanho'] >= 10]
dados = dados[dados['tamanho'] <= 150]
dados

In [ ]:
dados.drop(columns=['tamanho'], inplace=True)
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
dados.to_pickle('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/MaskedLanguageModeling/LipSet/LipSet_chunk.pkl')